## Get better at dask dataframes

In this lesson you will learn some good practices for dask dataframes and dealing with data in general.

## Parquet is where is at!!

You will learn the advantages of working with the parquet data format, and using the Uber/Lyft dataset you will learn to troubleshoot the nuances of working with real data. 


### Work close to your data

To get started when you are working with data that is in the cloud it's always better to work close to your data, to minimize the impact of IO networking. 

In this lesson, we will use coiled clusters that will be created on the same region that our datasets are stored. (the region is `"us-east-2"`)

**NOTE:**
If you do not have access to a coiled cluster you, can follow along just make sure you use the smaller dataset (use the `"0.5GB-"` ones). 

## Parquet vs CSV

Most people are familiarized with csv files, but when it comes to working with data, working with parquet can make a big difference. The Parquet file format is column-oriented and it's designed to efficiently store and retrieve data. 

### Small motivation example: 
Let's see an example where we compare reading the same data but in one case it is stored as `csv` files, while the other as `parquet` files. 

In [ ]:
data ={"0.5GB-csv": "s3://coiled-datasets/h2o-benchmark/N_1e7_K_1e2/*.csv",
       "0.5GB-pq": "s3://coiled-datasets/h2o-benchmark/N_1e7_K_1e2_parquet/*.parquet",
       "5GB-csv": "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2/*.csv",
       "5GB-pq": "s3://coiled-datasets/h2o-benchmark/N_1e8_K_1e2_parquet/*.parquet",}

In [ ]:
import coiled
from dask.distributed import Client
import dask.dataframe as dd

In [ ]:
%%time
cluster = coiled.Cluster(name="dask-tutorial", #RETHINK NAME ADD RANDOM UUID
                        n_workers=10,
                        package_sync=True,
                        backend_options={"region_name": "us-east-2"},
                        );

## maybe use mi6 instead, the default ones are slower...

In [ ]:
client = Client(cluster)
client

In [ ]:
ddf_csv = dd.read_csv(data["5GB-csv"], storage_options={"anon": True})
ddf_pq = dd.read_parquet(data["5GB-pq"], storage_options={"anon": True})
#dd.read_parquet(data["5GB-pq"], storage_options={"anon": True})

In [ ]:
ddf_csv

In [ ]:
ddf_pq

In [ ]:
%%time
ddf_csv.groupby("id1").agg({"v1": "sum"}).compute()

In [ ]:
%%time
ddf_pq.groupby("id1").agg({"v1": "sum"}).compute()

Notice that the `parquet` version without doing much it is already ~5X faster. 

Let's take a look at the memory usage as well as the `dtypes` in both cases.

In [ ]:
## memory usage for 1 partition
ddf_csv.partitions[0].memory_usage(deep=True).compute()

In [ ]:
ddf_pq.partitions[0].memory_usage(deep=True).compute()

In [ ]:
client.shutdown()

### Uber/Lyft data transformation

In the example above we quickly saw that the format in which the data is saved already makes a big difference. But there so much to exploit about the parquet file format. 

Let's work with the data from [High-Volume For-Hire Services](https://www.nyc.gov/site/tlc/businesses/high-volume-for-hire-services.page)



In [ ]:
import s3fs

s3 = s3fs.S3FileSystem()
files = s3.glob("nyc-tlc/trip data/fhvhv_tripdata_*.parquet")
files[:3]

In [ ]:
len(files)

In [ ]:
#not sure where the data is but I will write to a bucket in us-east-2
cluster = coiled.Cluster(
    n_workers=10,
    name="nyc-uber-lyft",
    package_sync=True,
    backend_options={"region": "us-east-2"}, 
    worker_memory="64 GiB", #we know we need a lot of memory from experience
)

In [ ]:
client = Client(cluster)
client

## Inspect the data

In [ ]:
client.restart()

In [ ]:
import dask

In [ ]:
ddf = dd.read_parquet(
    "s3://nyc-tlc/trip data/fhvhv_tripdata_*.parquet",
)
ddf

In [ ]:
#inspect memory usage of 1 partition
ddf.partitions[0].memory_usage(deep=True).compute().apply(dask.utils.format_bytes)

In [ ]:
#inspect dtypes
ddf.dtypes

## Challenges

As you can see, the partitions are very big, and the data types are inefficient.

## Recommendations and best practices:
**Partition size**

In general we aim for ~100MB (in memory) per partition. 

**dtypes**

- Avoid object types for strings: use `"string[pyarrow]"`
- Reduce int/float representation if possible
- Use categorical dtypes when possible.

### Create conversions dictionary

In [ ]:
import pandas as pd

In [ ]:
conversions = {}
for column, dtype in ddf.dtypes.items():
    if dtype == "object":
        conversions[column] = "string[pyarrow]"
    if dtype == "float64":
        conversions[column] = "float32"
    if dtype == "int64": 
        conversions[column] = "int32"
    if "flag" in column:
        conversions[column] = pd.CategoricalDtype(categories=["Y", "N"])
    if column == "airport_fee":
        conversions[column] = "float32"  #noticed that this has floats and the <NA> is making it an object
conversions

In [ ]:
ddf = ddf.astype(conversions)
ddf = ddf.persist()

In [ ]:
ddf.partitions[0].memory_usage(deep=True).compute().apply(dask.utils.format_bytes)

In [ ]:
dask.utils.format_bytes(
    ddf.partitions[0].memory_usage(deep=True).compute().sum()
)

### Repartition

In [ ]:
ddf = ddf.repartition(partition_size="128MB").persist()

In [ ]:
dask.utils.format_bytes(
    ddf.memory_usage(deep=True).compute().sum()
)

In [ ]:
ddf.npartitions

## Sort and one-day partitioning

In [ ]:
ddf = ddf.set_index("request_datetime").persist()

In [ ]:
ddf.divisions[:5]

Look like they are a bit longer than a day, we might as well repartition them witha  1-day frequency.

In [ ]:
ddf = ddf.repartition(freq="1d")

In [ ]:
ddf.divisions[:5]

In [ ]:
ddf.npartitions

In [ ]:
#Clever name for files when to_parquet
divisions = ddf.divisions

def name_file(index: int) -> str:
    return str(divisions[index].date()) + ".parquet"

name_file(0)

In [ ]:
ddf.to_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/", 
    name_function=name_file,
)

## Read data back

use_nullable_dtypes

In [ ]:
#client.restart()

In [ ]:
df = dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/", 
    use_nullable_dtypes=True
).astype({"hvfhs_license_num": "string[pyarrow]", 
         "dispatching_base_num": "string[pyarrow]",
         "originating_base_num": "string[pyarrow]",
         }).persist()
#df.dtypes

In [ ]:
df.dtypes

In [ ]:
df.hvfhs_license_num.dtype

In [ ]:
dask.utils.format_bytes(
    df.memory_usage(deep=True).sum().compute()
)

In [ ]:
Note:

Without pyarrow strings we get '~200GB'

In [ ]:
client.shutdown()

# On to a smaller cluster - let's do data analysis

Now we are at a stage that our whole dataset is ~75GB in memory. This is something we can work with in a smaller cluster. But also, when it comes to exploring data we do not necessarily need the whole data set.

One of the beauties of the parquet file format are:

- Column pruning: Get only the data of the column. 

In [ ]:
cluster = coiled.Cluster(name="uber-lyft-small",
                         n_workers=10, 
                         package_sync=True,
                         backend_options={"region_name": "us-east-2"},
)

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
df = dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/", 
    use_nullable_dtypes=True
).astype({"hvfhs_license_num": "string[pyarrow]", 
         "dispatching_base_num": "string[pyarrow]",
         "originating_base_num": "string[pyarrow]",
         })

In [ ]:
df.columns

In [ ]:
df_small = df[["base_passenger_fare", "driver_pay"]]

In [ ]:
df_small = df_small.persist()

In [ ]:
dask.utils.format_bytes(
    df_small.memory_usage(deep=True).sum().compute()
)
#'10.55 GiB'

In [ ]:
df_small.base_passenger_fare.sum().compute() / 1e9

In [ ]:
df_small.driver_pay.sum().compute() / 1e9

In [ ]:
df.head()